<div style="text-align: right"><i>Ivy Zhang<br>2023</i></div>

# How should you shuffle songs?

Almost a decade ago, Spotify had a [shuffling crisis](https://engineering.atspotify.com/2014/02/how-to-shuffle-songs/). Even though they were doing *theory optimal* shuffling, people weren't happy. Sure, the music was randomly shuffled, but it didn't **feel** randomly shuffled. Now why is that?

![](https://empiricalzeal.com/wp-content/uploads/2012/12/pinker-glow-worms-and-stars-plot.jpg)

Here are two distributions from Steven Pinker's *The Better Angels of Our Nature* and from [this lovely blog post](https://empiricalzeal.com/2012/12/21/what-does-randomness-look-like/). **Which one these is randomly distrbuted?**

Now perhaps after being primed with such a question, you're inclined to choose the left image, but our instinct is to think the right distribution is random even though it is much more uniform than random. It takes something more than randomness to spread points in a grid without forming any clusters. This desire to find patterns in random patterns isn't completely random, it's called [apophenia](https://en.wikipedia.org/wiki/Apophenia#:~:text=Apophenia%20has%20also%20come%20to,as%20can%20occur%20in%20gambling.), and it's a thing all us silly little humans do.

So what if we want to create distributions that *feel random*?


In [33]:
from collections        import Counter
from math               import ceil, sqrt
from random             import randint, random, uniform
from IPython.display    import clear_output, display_html

While we're here importing some packages, I'm also going to create an initial set of items that is split between `#` and `.` along with a utility to print out a grid of shuffled lists.

In [34]:
half_split_items = "########........"

In [35]:
def print_grid(items: str, shuffle_method, side_len: int):
    """Prints a grid of shuffled items using the given shuffle method."""
    for _ in range(side_len):
        for _ in range(side_len):
            print(shuffle_method(items) + "    ", end="")
        print("\n\n\n")

# (1) Perfectly Random Shuffles

To begin, let's try and generate some list of characters and truly randomly shuffle them. In what might be the luckiest event in all of computer science, one of the simplest way to solve a problem is also the most optimal!

From the original Spotify blog:
> I think [Fisher-Yates shuffle](https://en.wikipedia.org/wiki/Fisher%E2%80%93Yates_shuffle) is one of the most beautiful random algorithms and it’s amazing that such complicated problem can be solved in 3 lines of code in some programming languages. And this is accomplished using the optimal number of operations and optimal amount of randomness.

In [36]:
def fisher_yates_shuffle(items: str) -> str:
    items = list(items)
    n = len(items)
    for i in range(n-1, 0, -1):
        j = randint(0, i)
        items[i], items[j] = items[j], items[i]
    return "".join(items)

In [37]:
print_grid(half_split_items, fisher_yates_shuffle, 4)

..#.####...###..    #.##....#...####    ##..#.#..#.###..    ##.##.#....#.##.    



#...##..#.#.#.##    ...#..###.#.###.    .##....#..####.#    #.##.#.#..#.##..    



.###....####...#    #..#.#..##.#..##    ..#.###..##..#.#    #.##.#..#.#...##    



##.##.#.#....##.    ##.#....##.#..##    .##.#..###..##..    .#####..##.....#    





It's instantly clear that the same clustering effect we saw in the initial random grid shows up here. Sure, we've implemented an efficient random function, but most people probably don't want to hear a single artist upwards of 3 times in a row no matter how much we claim perfect randomness.

# (2) Human Perceived Randomness

For song shuffling, it doesn't seem to be *randomness* we want, but *diversity*. If you listen to a genre or artist once, then they shouldn't come on again. So rather than our initial approach of shuffling songs randomly, let's instead try spreading them out uniformly.

From the original Spotify blog post, they mention dividing items out on a line with some random offset, and I've tried my best to replicate that idea here in python.

In [38]:
def human_shuffle(items: str) -> str:
    parts = Counter(items)
    line_length = 1.00
    item_tuples = []
    for char, cnt in parts.items():
        offset = random()
        base_step = line_length / cnt
        for i in range(cnt):
            item_tuples.append((char, offset))
            offset += uniform(base_step - 0.05, base_step + 0.05)
            if (offset > line_length):
                offset -= line_length
    item_tuples.sort(key=lambda pair: pair[1])
    return "".join(c[0] for c in item_tuples)


In [39]:
print_grid(half_split_items, human_shuffle, 4)

#.#.##..##.#...#    #.##.#.#.#.#.#..    .##.##.#..#..#.#    .#.#.##..#.#.#.#    



.#.#.#.#..#.##.#    #..##..#.##.#..#    #..#.#..#.#.##.#    .#.#.##.#.#..#.#    



.#.#..#.##.#.##.    #.#.#..#.#.##.#.    #.#.#.#.#..##.#.    #.##..##.#.#..#.    



..###..#.#.#.#.#    .##.#..#.#.#.#.#    .#.#.#.#..#.#.##    .##..#.#.#.#.#.#    





Even with just the two genre playlist, you can clearly see what this offset + even distribution is doing to the songs. Songs are much more evenly distributed, so even though this might not be *random*, it'll feel random to the listener.

# (3) A Final Test

I'm not sure there's an objective way to measure these two things, but just for curiousity's sake, I've created a list with more categories of items to see how well the two algorithms split songs up. I'm sure that Spotify's shuffle is a lot more advanced than the simple algorithm I've coded up, but it's cool to see how even something so simple can come up with a passable reordering.

In [40]:
more_real_playlist = "#######@@@@@@...%%"

In [41]:
def compare_shuffles(items: str, side_len: int):
    print("Random Shuffle:\n")
    print_grid(items, fisher_yates_shuffle, side_len)

    print("\nHuman Shuffle:\n")
    print_grid(items, human_shuffle, side_len)

In [42]:
compare_shuffles(more_real_playlist, 4)

Random Shuffle:

#@.@@.%#@##@.##@#%    %@#@.###@%##..@@#@    @###@##@%..%#@.#@@    ##%#@.@#.%##@@.#@@    



@#.###@#%#.@.%@@@#    ##@%.#.@#@@@.###@%    #.@#.#@#%@@%###.@@    ##.@@.#@#%##@#%@@.    



@##.@.@#.%###@%#@@    #%.@@@###.@%#@@##.    @#.#@@#@@..###%@#%    #@@#.@@@@%##.#.#%#    



.@@#.%#@.%@@#@####    .##@@@@###.@@.%##%    #@@###.%@@#@@.#%.#    @@%@.#..##%@#@@###    




Human Shuffle:

#.@#@#.@%#@#@.#%#@    #@#%.@#@#.#@%#@#@.    @#.#@@#%.@##@#.%@#    %@#@.#@#@%#.@##@.#    



#.@@%#@.##@#.@#%#@    %#@@.##@#@.#%@#.@#    @#.%#@#@#.@#%@#.@#    @.#@#@%#.@##@#.@%#    



#@#@.#@%.#@#@#.#@%    #@%.#@#@#.#@%#@#.@    @#.@@#%#.@#@##.@%#    ##@%@.#@#@#.%#@#.@    



.@#%@#.@#@#.@%##@#    #.@##@.#@%#@#.@#@%    @##%@.#@#@#.@%#@#.    %#@#.@#@#.@%#@##.@    





This was largely inspired by Peter Norvig's [pytudes](https://github.com/norvig/pytudes) (you should definitely go check them out)! I'm sure there's room for improvement, both in code and writing, so if you have any ideas of how to improve this, send them my way on X (AKA Twitter) [@_ivyzhang](https://twitter.com/_ivyzhang)!